In [ ]:
import os
import zipfile

import branca
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
from fastkml import kml
from general_fun import *

In [ ]:
database_use = False

In [ ]:
if database_use:
    engine, conn = connect_database()

In [ ]:
company_colors = {
    "Alkyon Hydraulic Consultancy & Research": "green",
    "Heijmans": "darkred",
    "Koop Tjuchem": "lightgray",
    "MNO Vervat": "orange",
    "Seignette": "lightblue",
    "Van Oord": "darkblue",
}

company_logo = {
    "Alkyon Hydraulic Consultancy & Research": "Alkyon_Logo.png",
    "Heijmans": "Heijmans_Logo.png",
    "Koop Tjuchem": "koop_logo.png",
    "MNO Vervat": "mno_vervat_logo.png",
    "Seignette": "Heijmans_Logo.png",
    "Van Oord": "RVOlogo.png",
}

companies = list(company_colors.keys())

In [ ]:
# read KML content
kmz = zipfile.ZipFile(os.path.join(os.getcwd(), "gis", "Werken.kmz"), "r")  # extract zip file first, then read kmz file inside the extracted folder
kml_content = kmz.open("doc.kml", "r").read()  # kml content

# create KML object
k = kml.KML()
k.from_string(kml_content)

# read features from docs to folders to records and then extract geometries - in my case, Shapely points
docs = list(k.features())
folders = []
for d in docs:
    folders.extend(list(d.features()))

records = []
for f in folders:
    records.extend(list(f.features()))

geoms = [element.geometry for element in records]
names = [element.name for element in records]
descriptions = [element.description for element in records]

In [ ]:
def remove_duplicates(x):
    x = list(set(x.split(", ")))
    x = sorted(x)
    x = ", ".join(x)
    return x

In [ ]:
werken = pd.read_excel(
    os.path.join(os.getcwd(), "csv", "werken.ods"),
    engine="odf",
    converters={"date_start": str, "date_end": str, "project_number": str, "year": str, "Tools": str, "Type": str},
)
werken.columns = map(str.lower, werken.columns)

try:
    werken["date_start"] = pd.to_datetime(werken["date_start"], format="ISO8601", utc=False)
    werken["date_end"] = pd.to_datetime(werken["date_end"], format="ISO8601", utc=False)
except:
    werken["date_start"] = pd.to_datetime(werken["date_start"], format="%Y-%m-%d", utc=False)
    werken["date_end"] = pd.to_datetime(werken["date_end"], format="%Y-%m-%d", utc=False)

werken["id"] = werken.index + 1

werken["name"] = werken["name"].str.strip()

werken["years"] = werken.groupby(["name"])["year"].transform(lambda x: ", ".join(x))
werken["years"] = werken["years"].apply(lambda x: remove_duplicates(x))
werken["year"] = werken["year"].astype(int)

werken["Tools"] = werken["tools"].fillna("").astype(str)
werken["tools_all"] = werken.groupby(["name"])["Tools"].transform(lambda x: ", ".join(x))
werken["tools_all"] = werken["tools_all"].apply(lambda x: remove_duplicates(x))

werken.drop(columns="Tools", inplace=True)

werken["days_total"] = werken.groupby(["name"])["days"].transform("sum").astype(int)

In [ ]:
werken.head(50)

In [ ]:
projects = werken.drop_duplicates(keep="first", subset=["name"])

In [ ]:
df = gpd.read_file(os.path.join(os.getcwd(), "gis", "project_locations.shp"))
df.columns = map(str.lower, df.columns)
df.rename(
    columns={
        "descript": "description",
    },
    inplace=True,
)
df.drop(columns=["description", "icon", "elevation"], inplace=True)
df["latitude"] = df.apply(lambda x: x["geometry"].centroid.coords.xy[1][0], axis=1)
df["longitude"] = df.apply(lambda x: x["geometry"].centroid.coords.xy[0][0], axis=1)

df["company"] = [x.rsplit(" - ", 1)[-1] for x in df["name"]]
df["name"] = [x.rsplit(" - ", 1)[0] for x in df["name"]]

df["company_color"] = df["company"].map(company_colors)
df["company_logo"] = df["company"].map(company_logo)

geoms = [[point.xy[1][0], point.xy[0][0]] for point in df.geometry]

In [ ]:
werken = werken.merge(df[["name", "latitude", "longitude"]], how="left", on="name").reindex(
    columns=[
        "id",
        "name",
        "project_number",
        "company",
        "year",
        "date_start",
        "date_end",
        "days",
        "city",
        "country",
        "vessels",
        "latitude",
        "longitude",
        "description",
        "type",
        "tools",
    ]
)

if database_use:
    werken.to_sql("projects", conn, index=False, if_exists="replace", index_label="id")

In [ ]:
markers = df.merge(
    projects[["name", "years", "city", "country", "description", "vessels", "project_number", "type", "tools_all", "days_total"]],
    how="left",
    on="name",
)

In [ ]:
markers.head(50)

In [ ]:
if database_use:
    sql = """select * from home.house"""
    house = pd.read_sql(sql, conn)
else:
    house = pd.read_pickle("house.pkl")

geometry = gpd.points_from_xy(house["longitude"], house["latitude"])
geo_house = gpd.GeoDataFrame(house, geometry=geometry)
geo_house_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_house.geometry]

In [ ]:
mapit = folium.Map(location=np.array(house.loc[house["location"].isin(["Boskoop"]), ["latitude", "longitude"]])[0], tiles=None, zoom_start=9)

folium.TileLayer("openstreetmap", name="OpenStreet Map").add_to(mapit)
folium.TileLayer(
    "https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles &copy; Esri &mdash; National Geographic, Esri, DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA," "ESA, METI, NRCAN, GEBCO, NOAA, iPC",
    name="Nat Geo Map",
).add_to(mapit)
folium.TileLayer(
    "https://server.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles &copy; Esri &mdash; National Geographic, Esri, DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA," "ESA, METI, NRCAN, GEBCO, NOAA, iPC",
    name="World Imagery",
).add_to(mapit)

legend_txt = '<span style="color: {col};">{txt}</span>'

legend = {}
for cpy in companies:
    legend[cpy] = folium.FeatureGroup(name=legend_txt.format(txt=cpy, col=company_colors[cpy])).add_to(mapit)

for jj, coordinates in enumerate(geo_house_list):
    mapit.add_child(folium.Marker(location=coordinates, tooltip=f'<b>{house["location"][jj]}</b>', icon=folium.Icon(color="red", icon="home")))

for jj, geom in enumerate(geoms):
    html = popup_html(markers.iloc[jj, :])
    iframe = branca.element.IFrame(html=html, width=510, height=280)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)

    fm = folium.Marker(
        location=(markers["geometry"][jj].y, markers["geometry"][jj].x),
        tooltip=markers["name"][jj],
        popup=popup,
        icon=folium.Icon(color=markers["company_color"][jj], icon="person-digging", prefix="fa"),
    )

    fm.add_to(legend[markers["company"][jj]])

mapit.add_child(folium.LayerControl(position="topright", collapsed=True, autoZIndex=True))
mapit.save(os.path.join("/home/wcn/GitHub/py-home", "projects.html"))
mapit